<a href="https://colab.research.google.com/github/whoamiwhress/blank/blob/main/minecraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minecraft on Colab  
For those people who run 💩 on Google Colaboratory.  
Because why NOT turn a machine learning notebook into a literal block party. 🎉  
🧱✨ Powered by ngrok, chaos, and the tears of sysadmins.  

In [ ]:
#@title 🚀 Minecraft Server on Colab (Main Launch Cell v2)
#@markdown ## Minecraft on Colab
#@markdown _Blocky chaos in the cloud, powered by Google and Gen Z._
#@markdown ---
#@markdown 🔄 **Choose Version Type**
release_type = "release" #@param ["release", "snapshot"]
#@markdown 🔓 **Use playit.gg for TCP tunnel** (Recommended!)
use_playit = True #@param {type:"boolean"}

import subprocess
import sys
import re
import os

# ⚙️ Install packages & playit.gg
def auto_install(package):
    try:
        __import__(package)
    except ImportError:
        print(f"[⚠️] Missing '{package}', installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

for pkg in ["httpx"]:
    auto_install(pkg)

print("[🛠️] Installing playit.gg for TCP tunnel 🔌...")
subprocess.run("curl -SsL https://playit-cloud.github.io/ppa/key.gpg | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/playit.gpg >/dev/null", shell=True)
subprocess.run('echo "deb [signed-by=/etc/apt/trusted.gpg.d/playit.gpg] https://playit-cloud.github.io/ppa/data ./" | sudo tee /etc/apt/sources.list.d/playit-cloud.list', shell=True)
subprocess.run("sudo apt update && sudo apt install playit -y", shell=True)

import httpx

# 🔥 Get latest version
def get_latest(release_type="release"):
    url = "https://launchermeta.mojang.com/mc/game/version_manifest_v2.json"
    try:
        res = httpx.get(url, timeout=10.0)
        res.raise_for_status()
        data = res.json()
        if release_type not in ["release", "snapshot"]:
            print(f"[⚠️] Invalid release_type '{release_type}', defaulting to 'release'")
            release_type = "release"
        latest_version = data["latest"][release_type]
        print(f"[🆕] Latest {release_type}: {latest_version}")
        return latest_version
    except Exception as e:
        print(f"[X] Failed to fetch version: {e}")
        return None

# 📰 Get article URL
def get_article_url(version):
    if re.match(r"..w..+", version):
        return f"https://www.minecraft.net/en-us/article/minecraft-snapshot-{version}"
    elif "pre" in version:
        slug = version.replace('.', '-').replace(' ', '-').lower()
        return f"https://www.minecraft.net/en-us/article/minecraft-{slug}"
    else:
        slug = version.replace('.', '-')
        return f"https://www.minecraft.net/en-us/article/minecraft-java-edition-{slug}"

# 🎯 Grab server JAR
def fetch_server_jar(version):
    url = get_article_url(version)
    print(f"[🔗] Scanning article: {url}")
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64)",
        "Accept": "text/html",
        "Accept-Language": "en-US,en;q=0.5",
    }
    try:
        with httpx.Client(http2=True, headers=headers, timeout=15.0) as client:
            res = client.get(url)
            if "Minecraft server jar" in res.text:
                match = re.search(r'https://piston-data\.mojang\.com/v1/objects/[a-f0-9]+/server\.jar', res.text)
                if match:
                    print(f"[✅] JAR found: {match.group()}")
                    return match.group()
    except httpx.RequestError as e:
        print(f"[X] HTTP error: {e}")
    return None

# ⬇️ Download the JAR
def download_jar(jar_url, dest_path="server.jar"):
    try:
        print(f"[⬇️] Grabbing server JAR → {dest_path}")
        res = httpx.get(jar_url)
        with open(dest_path, "wb") as f:
            f.write(res.content)
        print(f"[✅] JAR download complete.")
    except Exception as e:
        print(f"[X] Download failed: {e}")

# 🎮 Start MC server
def start_server():
    if not os.path.exists("eula.txt"):
        with open("eula.txt", "w") as f:
            f.write("eula=true\n")
    print("[🧱] Launching Minecraft server...")
    subprocess.Popen(["java", "-Xms1G", "-jar", "server.jar", "nogui"])

import threading

# 🌐 Launch playit tunnel with live stdout stream
def start_playit():
    print("[🌍] Launching playit.gg tunnel (LIVE logs)...")
    def stream_playit():
        proc = subprocess.Popen(
            ["playit"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )
        for line in proc.stdout:
            print(f"[playit] {line.strip()}")

    threading.Thread(target=stream_playit, daemon=True).start()

# 🎬 Main launch
version = get_latest(release_type)
print(f"[🎯] Using version: {version}")
jar_url = fetch_server_jar(version)

if jar_url:
    download_jar(jar_url)
    start_server()
    if use_playit:
        start_playit()
    else:
        print("[⚠️] No tunnel started. Set `use_playit = True` if you want public access.")
else:
    print("[💥] Server JAR not found. Go cry in redstone.")
